#

infile = open('samplePosforPeaks', 'rb')
testarray = pickle.load(infile)
extract_boundaries(testarray, height = 0.001)

# Import all the libraries that we need

In [ ]:
import csv
import datetime
import fnmatch
import glob
import itertools
from itertools import groupby
import math
import matplotlib.cbook
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import pandas as pd
import pickle
import platform
from pylab import rcParams
import re
import scipy
from scipy import stats
from scipy.signal import find_peaks
import statistics
import warnings
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)

# Define #ROOT

In [ ]:
###Set the notebook screen ratio for jupyter
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:85% !important; }</style>"))

notebookPath=os.path.dirname(os.path.realpath("__file__"))
#pathToProcessingNotebooks=os.path.join(os.path.split(notebookPath)[0],"load_preprocess_rat")
#os.chdir(pathToProcessingNotebooks)   
#%run loadRat_documentationmod.ipynb
#%run plotRat_documentation_1_GeneralBehaviormod.ipynb

#### DEFINE ROOT
if platform.system()=='Linux':
    root="/home/david/Desktop/DATARUN/test0820"
    savePath="/home/david/Desktop/Save"
else:
    #mac = 'Darwin'
    root="/Users/tom/Desktop/DATARUN/test0820" #current : test0826
    savePath="/Users/tom/Desktop"#os.path.join(root,"ALLRATS","Fig")
print("Root: ", root)

retval = os.getcwd()
print("Current working directory: %s" % retval)#print("Notebook Path: ", notebookPath)
print("Save Path: ", savePath)   
print("Path to data is: %s"%root)
print("")
print("importing done")

# Define utility functions

In [ ]:
#conversion functions
def inch2cm(value):
    return value/2.54
def cm2inch(value):
    return value*2.54
def px2cm(value):
    return value/13.5

#function to split lists --> used to split the raw X position array into smaller arrays (runs and stays). Later in the code we modify the array and change some values to 0, which will be used as cutting points.
def split_a_list_at_zeros(List):
    return [list(g) for k, g in groupby(List, key=lambda x:x!=0) if k]

#function to open and read from the .position files using pandas, specify the path of the file to open, the column that you want to extract from, and the extension of the file
def read_csv_pandas(path,Col=None,header=None):
    #verify that the file exists
    if not os.path.exists(path):
        print("No file %s"%path)
        return []
    try:
        #open the file
        csvData=pd.read_csv(path,header=header,delim_whitespace=True)
    except ValueError:
        print("%s not valid (usually empty)"%path)
        return []
        #verify that the column that we specified is not empty, and return the values
    if Col is not None:
        return csvData.values[:, Col[0]]
    else:
        return csvData

# function to cut the 30min session (position plus time) into N bins of equal lenght, specify the data to cut (position), data template(we make the bins of a certain lenght so we use time), bot is where we start (usually 0, but can start later if we don't want the start), end is the end of what we want (usually the end of the session), binNumber is the number of bin that we want.
def bin_session(data_to_cut, data_template,  bot, end, binNumber):
    output={}
    for count in range(0, binNumber):
        start_of_bin = count*end/binNumber
        end_of_bin = count*end/binNumber+end/binNumber
        output[count] = data_to_cut[animal, session][np.logical_and(data_template[animal, session] >= start_of_bin, data_template[animal, session] < end_of_bin)]
        #print(boot, toop, output, len(output))    
    return output

#function to read the parameters for each rat for the session in the behav.param file. Specify the name of the parameter that you want to get from the file and optionally the value type that you want. File path is not an option, maybe change that.
def read_params(paramName, valueType=str):
    #define path of the file
    behav = root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".behav_param"
    #check if it exists
    if not os.path.exists(behav):
        print("No file %s" % behav)  
    #check if it is not empty
    #if os.stat(behav).st_size == 0:
        #print("File empty %s" % behav)
    #open the file
    with open(behav, "r") as f:
        #scan the file, if the name of the parameter is here, get the value 
        #scan the file
        for line in f:
            #if you look for a string do that
            if valueType is str:
                #find the line in which you have your parameter
                if paramName in line:
                    #get the last value of the line [-1], values are separated with _blanks_ with the .split() function
                    return int(line.split()[-1])  
                    #same if you want a float value, just convert the value
            if valueType is float: 
                if paramName in line:
                    return float(line.split()[-1])
            #can add other value type if needed (boolean, etc. )

# Load Data
### Data is loaded in one dictionary per variable (time, position, etc.) and have [animal, session] as keys, so we have a set of data for each session of each rat
### This cell loads all the behavioral data from the files created with labview

In [ ]:
# initialise arrays and dicts

rawTime={}
rawPositionX={}
rawPositionY={}
extractPositionX = {}
extractPositionY = {}
extractTime = {}
extractLickLeft = {}
extractLickRight = {}
rawLickLeft1X = {}
rawLickRight1X = {}
rawLickLeft1Y = {}
rawLickRight1Y = {}

timeBins = 6
binPositionX={}
binTime={}

#create the list of the animals and sessions
animalindex=1
animalList = [os.path.basename(path) for path in sorted(glob.glob(root+"/Rat*"))]
for animal in animalList:
    print("Animal %s"%animal)
    print("%s out of %s" %(animalindex,len(animalList)))
    print("**********")
    animalindex += 1
    #Get the list of all session
    sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
    sessionList=sorted(sessionList)
    nbSession=len(sessionList)
    
    #loop through animals and sessions
    for index, session in enumerate(sessionList):
        print("Session %s/%s: %s"%(index+1,nbSession,session), "['" +animal+ "\',\'" +session + "\']")
        acquisitionCutOff = read_params("sessionDuration")
 
        #load data for each session
        #data loaded : time array, position of the animal X and Y axis, Licks to the left and to the right
        extractTime[animal, session]      = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"),Col=[3])#old format = 5
        extractPositionX[animal, session] = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"),Col=[4])#old format = 6
        extractPositionY[animal, session] = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"),Col=[5])
        extractLickLeft[animal, session]  = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"),Col=[6])
        extractLickRight[animal, session] = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"),Col=[7])
        #directly cut the data at the end of the session  
        rawTime[animal, session]       = extractTime[animal, session][extractTime[animal, session] <= acquisitionCutOff]
        rawPositionX[animal, session]  = extractPositionX[animal, session][extractTime[animal, session] <= acquisitionCutOff]
        rawPositionY[animal, session]  = extractPositionY[animal, session][extractTime[animal, session] <= acquisitionCutOff]
        rawLickLeft1X[animal, session]  = extractLickLeft[animal, session][extractTime[animal, session] <= acquisitionCutOff]
        rawLickLeft1Y[animal, session]  = extractLickLeft[animal, session][extractTime[animal, session] <= acquisitionCutOff]
        rawLickRight1X[animal, session] = extractLickRight[animal, session][extractTime[animal, session] <= acquisitionCutOff]
        rawLickRight1Y[animal, session] = extractLickRight[animal, session][extractTime[animal, session] <= acquisitionCutOff]

        #create the N time bins
        binTime[animal, session] = bin_session(rawTime, rawTime, 0, acquisitionCutOff, timeBins)
        binPositionX[animal, session] = bin_session(rawPositionX, rawTime, 0, acquisitionCutOff, timeBins)
        
        print()   
#clear_output()

# LICKSTUFF TO DO
# Work in progress

In [ ]:
#rawLickLeft1X[animal, session][1000:2000]

In [ ]:
for animal in animalList:
    sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
    for session in sessionList: 
        #when true use rawpositionX, when false use None
        rawLickLeft1X[animal, session][rawLickLeft1X[animal, session]==True] = rawPositionX[animal, session][rawLickLeft1X[animal, session]==True]
        rawLickLeft1X[animal, session][rawLickLeft1X[animal, session]==False] = None    
        rawLickRight1X[animal, session][rawLickRight1X[animal, session]==True] = rawPositionX[animal, session][rawLickRight1X[animal, session]==True]
        rawLickRight1X[animal, session][rawLickRight1X[animal, session]==False] = None

        #same for Y position
        rawLickLeft1Y[animal, session][rawLickLeft1Y[animal, session]==False] = None
        rawLickLeft1Y[animal, session][rawLickLeft1Y[animal, session]==True] = rawPositionY[animal, session][rawLickLeft1Y[animal, session]==True]
        rawLickRight1Y[animal, session][rawLickRight1Y[animal, session]==False] = None
        rawLickRight1Y[animal, session][rawLickRight1Y[animal, session]==True] = rawPositionY[animal, session][rawLickRight1Y[animal, session]==True]

In [ ]:
"""for animal in animalList:
    sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
    for session in sessionList: 
        for i in rawLickLeft1X[animal, session]:
            if i is not None:
                if i >= 170:
                    print(i)
                    i = None


        
        for i in rawLickRight1X[animal, session]:
            if i is not None:
                if i <=880:
                    print(i)
                    i = None
        print()

        
        
        #rawLickRight1X[animal, session]"""
        

# PARAMS
### Extract parameters from the behav.param files and create a dictionnary for each rat with every value
### * sessionDuration : duration of the session 
### * acqPer : number of acquisition per second in labview // not used here
### * waterLeft : (weight of the cup + water - weight of the cup) / 10 --> Weight of a drop on the Left  
### * waterRight : idem on right  
### * treadmillSize : size of the treamdmill for the session #remove ?     
### *      "boundaryLeft"   #remove
### *      "boundaryRight"  #remove
### *      "middle"         #removed ?
### * weight : Weight of each rat just before the session 
### * lastWeightadlib : Weight of each rat when the water bottles are removed
### * lastDayadlib : Last day with ad lib water
### * daysSinceadlib : compute the number of days have elapsed since "lastDayadlib"
### * boundaries : initialise the boundary array, that is computed later -> define where (on the X axis) the rat is drinking
###
### * Create rat markers : used in plots, define a marker for each rat (e.g. in this experiment male and female symbols), and a color for each rat

In [ ]:
params = {}
params[animal, session] = []
for animal in animalList:
    sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
    for session in sessionList: 
        #create a dictionary and read the params wor each rat
        params[animal, session] = {
        "sessionDuration" : read_params("sessionDuration"),
        "acqPer" : read_params("acqPer"),
        "waterLeft" : read_params("waterLeft", valueType = float) - read_params("cupWeight", valueType = float),   
        "waterRight" : read_params("waterRight", valueType = float) - read_params("cupWeight", valueType = float),   
        "treadmillSize" : read_params("treadmillSize"), #remove ?     
        "boundaryLeft" : read_params("boundaryLeft"),  #remove
        "boundaryRight" : read_params("boundaryRight"), #removed
        "middle"       : (read_params("boundaryLeft") + read_params("boundaryRight"))/2,  #removed ?
        "weight" : read_params("ratWeight"), 
        "lastWeightadlib" : read_params("ratWeightadlib"),
        "lastDayadlib" : read_params("lastDayadlib")
        }

        lastDayadlib = str(datetime.datetime.strptime(str(read_params("lastDayadlib")), "%Y%m%d").date())
        match = re.search(r'\d{4}_\d{2}_\d{2}', session)
        experimentDay = str(datetime.datetime.strptime(match.group(), '%Y_%m_%d').date())
        lastDayadlib = datetime.date(int(lastDayadlib[0:4]), int(lastDayadlib[5:7]), int(lastDayadlib[8:10]))
        experimentDay = datetime.date(int(experimentDay[0:4]), int(experimentDay[5:7]), int(experimentDay[8:10]))
        daysSinceadlib = experimentDay-lastDayadlib
        params[animal, session]["daysSinceadLib"] = daysSinceadlib.days
        #labview ok 27/07/2020 before nOk #format behavparam ?
        #catchup manual up to 27/07
        params[animal, session]["boundaries"] = []


In [ ]:
rat_markers={}
rat_markers[animal] = {}
palette = ['r', 'm', 'y', 'c', 'r', 'm', 'y', 'c']
index = 0
for animal in animalList:
    # if the animal is F use Female symbol and 1 color per F rat
    if fnmatch.fnmatch(animal, 'RatF*'):
        rat_markers[animal]=[palette[index], "$\u2640$"]
        index+=1
    elif fnmatch.fnmatch(animal, 'RatM*'):
        #same for males, same colors but different symbol
        rat_markers[animal]=[palette[index], "$\u2642$"]
        index+=1  
        #if rat is not F or M (Rat000) use black and ? 
    elif fnmatch.fnmatch(animal, 'Rat00*'):
        rat_markers[animal]=[palette[index], "$\u2426$"]
        index+=1 
    else:
        print("error, bypass? Y/N")
        answer = input()
        if answer== 'Y':
            rat_markers[animal]= ['k',"$\u2426$"] 
        else:
            print("do nothing")

In [ ]:
print(params)

# BASE TRAJECTORY
### This first part is about plotting the X trajectory of the animals. Lick data was added on the base trajectory in a later version.

In [ ]:
# This function plots the base trajectory of the rat. Parameters are time : time data, position : X position data, lickL/R, lick data, maxminstep for x and y axis, color and marker of the plot, width of the axis, and x y labels
def plot_BASEtrajectory(time, position, lickLeft, lickRight, maxminstep, maxminstep2, color = [], marker = [], linewidth = [], xyLabels=["N","Bins"]):   
       
        plt.plot(time, position, color=color[0], marker=marker[0], linewidth = linewidth[0])
        plt.plot(time, lickLeft, color=color[1], marker=marker[1], markersize = marker[2])
        plt.plot(time, lickRight, color=color[1], marker=marker[1], markersize = marker[2])

        ax = plt.gca()
        #ax.set_xlim(maxminstep[:2])
        #configure ax xy span
        ax.set_xlim(maxminstep[0]-maxminstep[2],maxminstep[1]+maxminstep[2])
        ax.set_ylim(maxminstep2[0]-maxminstep2[2],maxminstep2[1]+maxminstep2[2])
        #configure xy style
        ax.set_xlabel(xyLabels[1],fontsize=12,labelpad=0)
        ax.set_ylabel(xyLabels[0],fontsize=12,labelpad=-1)
        ax.xaxis.set_ticks_position('bottom')
        ax.yaxis.set_ticks_position('left')
        ax.get_xaxis().set_tick_params(direction='out',pad=2)
        ax.get_yaxis().set_tick_params(direction='out',pad=2)
        #remove unnecessary axes
        ax.spines['top'].set_color("none")
        ax.spines['right'].set_color("none")
        return ax

In [ ]:
# this function computes the value of the resting point of the animals, that we define as the boundary to determine where a run starts
# params : data : X position array, animal and session that we analyse, height = parameter to define a peak
# explanation : if you look at the histogram of the X position of the animal, you notice that there are two peaks that represent a position in which the animal spends a lot of time (i.e., the position in which they all stop, the reward port). We use a kernel density estimate (KDE) to get a fit curve over our data (red line on the histogram), we then have to compute the two peaks of this curve and get their position. We then add a randomly selected 50 px inwards and define these points as our boundaries.
def extract_boundaries(data, animal, session, height = None):
    if type(data)==list:
        #check if the data is a list
        #compute KDE of the data
        #for current pickle example or if there is a bug somewhere, but can be removed if not in use
        kde = stats.gaussian_kde(data)
    else:
        kde = stats.gaussian_kde(data.tolist())
    #get the curve and compute peaks
    nb_samples = 14000
    samples = np.linspace(0, 1400, nb_samples)
    probs = kde.evaluate(samples)
    maxima_index = find_peaks(probs, height) 
    # if there is more than two peaks (e.g. an animal decides to stay in the middle of the treadmill), only keep the two biggest peaks (should be the extremities) and remove the extra peak/s if there is one or more
    if len(maxima_index[1]["peak_heights"])>=3:
        #if we remove one peak or more, the index of the peak is displayed when computing
        print("computing", animal, np.argmin(maxima_index[1]["peak_heights"]))
        index = np.argmin(maxima_index[1]["peak_heights"])
        maxima_index[1]["peak_heights"] = np.delete(maxima_index[1]["peak_heights"], index)
        extractedBounds = np.delete(maxima_index[0], index)
        leftBoundaryPeak = samples[extractedBounds[0]]
        rightBoundaryPeak = samples[extractedBounds[1]]
    else:
        print("computing", animal)
        leftBoundaryPeak = samples[maxima_index[0][0]]
        rightBoundaryPeak = samples[maxima_index[0][1]]
    
    return leftBoundaryPeak, rightBoundaryPeak, kde


# this function plots the peak that we just extracted
def plot_peak(data, animal, session, leftBoundaryPeak, rightBoundaryPeak, kde, maxminstep, maxminstep2, marker = [], xyLabels=["N","Bins"]):

    #fig, ax = plt.subplots(figsize=(3,6))
    bins = np.arange(1400)
    xx = np.linspace(0, 1400, 1400)
    xline1 = [leftBoundaryPeak, leftBoundaryPeak]
    xline2 = [rightBoundaryPeak, rightBoundaryPeak]
    yline = [0, 0.01]

    if platform.system()== 'Darwin':
        ax.hist(data, normed=True, bins=bins, alpha=0.3, orientation='horizontal') #bugged on linux, working on mac

    ax.plot(kde(xx), xx, color = 'r')
    ax.plot(yline, xline1, ":", color='k')
    ax.plot(yline, xline2, ":", color='k')

    ax.plot(yline, [xline1[0]+border, xline1[0]+border], ":", color='k', alpha = 0.5)
    ax.plot(yline, [xline2[0]-border, xline2[0]-border], ":", color='k', alpha = 0.5)

    ax.set_xlim(maxminstep[0]-maxminstep[2],maxminstep[1]+maxminstep[2])
    ax.set_ylim(maxminstep2[0]-maxminstep2[2],maxminstep2[1]+maxminstep2[2])
    ax.set_xlabel(xyLabels[1],fontsize=12,labelpad=0)
    ax.set_ylabel(xyLabels[0],fontsize=12,labelpad=-1)
    ax.spines['top'].set_color("none")
    ax.spines['left'].set_color("none")
    ax.spines['right'].set_color("none")
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    ax.get_xaxis().set_tick_params(direction='out',pad=2)
    ax.get_yaxis().set_tick_params(direction='out',pad=2)
    ax.axes.get_yaxis().set_visible(False)

    return ax

In [ ]:
leftBoundaryPeak = {}
rightBoundaryPeak = {}
kde = {}
for animal in animalList:
#for animal in animalList[0:2]: # 2 rats for quick testing
    sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
    for session in sessionList: 

        if params[animal, session]["boundaries"] == []: #skip computing peaks if there is already a value computed and stored in params (value is [] when setting up all params so when running notebook for the first time peaks are computed)
            border = 50
            leftBoundaryPeak[animal, session], rightBoundaryPeak[animal, session], kde[animal, session]= extract_boundaries(rawPositionX[animal, session], animal, session, height = 0.001)
            params[animal, session]["boundaries"] = [rightBoundaryPeak[animal, session] - border, leftBoundaryPeak[animal, session] + border]

        print("plotting", animal)
        fig = plt.figure(constrained_layout=True, figsize=(21,6))
        fig.suptitle(animal + " " + session)
        widths = [3, 18]
        heights = [6]
        rows = 1
        cols = 2

        spec = fig.add_gridspec(ncols=cols, nrows=rows, width_ratios=widths, height_ratios=heights)
        index =0
        for row in range(rows):
            for col in range(cols):
                ax = fig.add_subplot(spec[row, col])
                
                if index == 0:
                    plot_peak(rawPositionX[animal, session], animal, session, leftBoundaryPeak[animal, session], rightBoundaryPeak[animal, session], kde[animal, session],[0.02, 0, 0],[0,1400,0],  marker = [""], xyLabels=["Position (px)", "%"])
                    index+=1
                    
                elif index == 1:
                    plot_BASEtrajectory(rawTime[animal, session],rawPositionX[animal, session], rawLickLeft1X[animal, session], rawLickRight1X[animal, session], [0,1800,1],[0,1400,1],  color = ["b", "c"], marker = ["", "o", 1], linewidth = [0.5], xyLabels=["Position (cm)", "Time(s)"])
                    index+=1
       

In [ ]:
plt.figure(figsize = [20, 5])
for animal in animalList:
    sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
    for session in sessionList: 
        plt.figure(figsize = [30, 5])
        print(animal, session)
        plot_BASEtrajectory(rawTime[animal, session],rawPositionX[animal, session], rawLickLeft1X[animal, session], rawLickRight1X[animal, session], [0,1800,1],[0,1400,1],  color = ["b", "c"], marker = ["", "o", 1], linewidth = [0.5], xyLabels=["Position (cm)", "Time(s)"])

# Define functions
# extract runs and stays
# function to compute stats
# function to plot tracks
# function to plot distribution
# function to plot recap

In [ ]:
def extract_runsRightLeftFiltered(data, animal, session, boundary, limspeed=None):
    rawdata_run = {}
    rawdata_stay = {}
    good_speedRight_cms = []
    good_speedLeft_cms = []
    good_x_trackR = []
    good_x_trackL = []

    bad_speedRight_cms = []
    bad_speedLeft_cms = []
    bad_x_trackR = []
    bad_x_trackL = []
    all_speedRight_cms = [] 
    all_speedLeft_cms = []


    good_timeInRight = []
    good_timeInLeft = []
    good_x_stayR = []
    good_x_stayL = []

    bad_timeInRight = []
    bad_timeInLeft = []
    bad_x_stayR = []
    bad_x_stayL = []
    all_timeInRight = []
    all_timeInLeft = []

    #crop the complete track at the set boundaries -> keep only the runs and set the rest to 0
    rawdata_run[animal, session] = [e/13.5 if e <= boundary[0] and e>= boundary[1] else 0 for e in data[animal, session]] #conversion pixel to cm maybe do this better 
    # use the split a list function to create a new sublist when there is a 0, which we just set up before
    for subsplit in split_a_list_at_zeros(rawdata_run[animal, session]):
        #check if the subsplit starts on the left or the right -> determine if the animal is running left or right
        if subsplit[0] < (((boundary[0]+boundary[1])/2) /13.5):
            #check if the subsplit is ending on the other side -> determine if this is a run 
            if subsplit[-1] > (((boundary[0]+boundary[1])/2) /13.5):
                #calculate distance run as the distance between first and last value
                distRight_cm = abs(subsplit[0]-subsplit[-1]) 
                #calculate time as number of values (frames) * time per frame
                timeRight = len(subsplit) * 0.05
                #check if there is a limspeed : if no compute all the runs -> used to compute limspeed
                #second pass : there is a computed limspeed, use it to sort good and bad runs
                if limspeed[0] is not None:
                    #check if this is a good or bad run and assign im to the good or bad array accordingly
                    if distRight_cm/timeRight > limspeed[0]:
                        #good run array
                        good_speedRight_cms.append(distRight_cm/timeRight)
                        good_x_trackR.append(subsplit)     
                    else:
                        #bad run array
                        bad_speedRight_cms.append(distRight_cm/timeRight)
                        bad_x_trackR.append(subsplit)
                else:
                    #all runs array first pass
                    all_speedRight_cms.append(distRight_cm/timeRight)

        #same thing for the runs that go to the other side
        elif subsplit[0]>(((boundary[0]+boundary[1])/2)/13.5):
            if subsplit[-1]<(((boundary[0]+boundary[1])/2)/13.5):
                distLeft_cm = abs(subsplit[0]-subsplit[-1])
                timeLeft = len(subsplit) * 0.05
                if limspeed[1] is not None:
                    if distLeft_cm/timeLeft > limspeed[1]:
                        good_speedLeft_cms.append(distLeft_cm/timeLeft)   
                        good_x_trackL.append(subsplit)
                    else:
                        bad_speedLeft_cms.append(distLeft_cm/timeLeft)
                        bad_x_trackL.append(subsplit)
                else:
                    all_speedLeft_cms.append(distLeft_cm/timeLeft)

    # inverse crop, at the boundaries keep the distal parts, same process as above
    rawdata_stay[animal, session] = [e/13.5 if e >= boundary[0] or e<= boundary[1] else 0 for e in data[animal, session]]
    for subsplit in split_a_list_at_zeros(rawdata_stay[animal, session]):
        #first identify if the subsplit created is on the left or right by comparing to the middle
        if subsplit[0] > (((boundary[0]+boundary[1])/2) /13.5):
            # maybe add another filter i.e. out if less than 5 frames
            #same idea as above
            if limspeed[0] is not None:
                #add lenght (frames) of the subsplit * time per frame
                if len(subsplit)* 0.05 < limspeed[0]:
                    good_timeInRight.append(len(subsplit)* 0.05)#clean 
                    good_x_stayR.append(subsplit)
                else:
                    bad_timeInRight.append(len(subsplit)* 0.05)
                    bad_x_stayR.append(subsplit)
            else:
                all_timeInRight.append(len(subsplit)* 0.05)
                
        elif subsplit[0]<(((boundary[0]+boundary[1])/2)/13.5):
            #compute
            if limspeed[1] is not None:
                if len(subsplit)* 0.05 < limspeed[1]:
                    good_timeInLeft.append(len(subsplit)* 0.05)   
                    good_x_stayL.append(subsplit)
                else:
                    bad_timeInLeft.append(len(subsplit)* 0.05)
                    bad_x_stayL.append(subsplit)
            else:
                all_timeInLeft.append(len(subsplit)* 0.05)
   
    return good_speedRight_cms, good_speedLeft_cms, good_x_trackR, good_x_trackL, bad_speedRight_cms, bad_speedLeft_cms, bad_x_trackR, bad_x_trackL, all_speedRight_cms, all_speedLeft_cms, good_timeInRight, good_timeInLeft, good_x_stayR, good_x_stayL, bad_timeInRight, bad_timeInLeft, bad_x_stayR, bad_x_stayL, all_timeInRight, all_timeInLeft

In [ ]:
def statsrun(dataRight, dataLeft, animal):
    #define sitistical significance threshold
    sig_threshold = 0.05
    #compute average of speeds
    average_semToRight = [np.mean(dataRight), scipy.stats.sem(dataRight), np.median(dataRight)]
    average_semToLeft = [np.mean(dataLeft), scipy.stats.sem(dataLeft), np.median(dataLeft)]
    #check if the sample is from a normal distribution 
    #normaltest, if p<0.05, reject null hypothesis that sample comes from a normal distribution
    normalToRight = scipy.stats.normaltest(dataRight)
    normalToLeft = scipy.stats.normaltest(dataLeft)

    print(animal)
    #check if the two samples that we are comparing are from a normal distribution; if they both are perform a ttest, if at least one of them is not do a mannwhitney 
    if normalToLeft[1] > sig_threshold and normalToRight[1] > sig_threshold:
        #stat from ttest
        statValue = [scipy.stats.ttest_ind(dataRight, dataLeft)[1], "Ttest"]
    else: 
        #stat from mannwhitney
        statValue = [scipy.stats.mannwhitneyu(dataRight, dataLeft)[1], "MannWhitney"]

    #if the stat value (ttest or mannwhitney) is below statistical signifiance threshold show big arrow to the side they go the faster, print a *, else not
    if statValue[0] > sig_threshold:
        print("----------->", "\t", "<-----------")
    else:
        if average_semToRight[0] > average_semToLeft[0]:
            print("=>=>=>=>=>=>", "\t", "<-----------")
        else:
            print("----------->", "\t", "<=<=<=<=<=<=")
    #print and format the results
    print("%.2f" % average_semToRight[0],"\u00B1", "%.2f" % average_semToRight[1], "\t" ,"%.2f" % average_semToLeft[0],"\u00B1", "%.2f" % average_semToLeft[1], "     median R - L ", "%.2f" % average_semToRight[2], "%.2f" % average_semToLeft[2])
    print(statValue[0], "" if statValue[0] > sig_threshold else "*", statValue[1])
    print()

In [ ]:
def plot_tracks(animal, session, gooddataRight, gooddataLeft, baddataRight, baddataLeft, xylim, xyLabels=["Speed cm/s","Quantity", "Direction", "Speed(cm/s)"], title=[None], suptitle = [""], linewidth = 1): 
    plt.figure(figsize = [10, 5])
    plt.suptitle(suptitle[0], fontsize = suptitle[1])
    ax = plt.subplot(1, 2, 1)
    for i in gooddataRight:
        plt.plot(np.linspace(0, len(i)*0.05, len(i)), i, color='g' , linewidth=0.3)
    for k in baddataRight:
        plt.plot(np.linspace(0, len(k)*0.05, len(k)), k, color='r', linewidth=0.3)

    ax.set_title(title[0], fontsize = title[2])
    ax.set_xlabel(xyLabels[0], fontsize = xyLabels[3])
    ax.set_ylabel(xyLabels[1], fontsize = xyLabels[3])
    ax.set_xlim([xylim[0], xylim[1]])
    ax.set_ylim([xylim[2], xylim[3]])
    ax.spines['top'].set_color("none")
    ax.spines['right'].set_color("none")

    ax2 = plt.subplot(1, 2, 2)
    for j in gooddataLeft:
        plt.plot(np.linspace(0, len(j)*0.05, len(j)), j, color='g', linewidth=0.3)
    for l in baddataLeft:
        plt.plot(np.linspace(0, len(l)*0.05, len(l)), l, color='r', linewidth=0.3)

    ax2.set_title(title[1], fontsize = title[2])
    ax2.set_xlabel(xyLabels[0], fontsize = xyLabels[3])
    ax2.set_ylabel(xyLabels[1], fontsize = xyLabels[3])
    ax2.set_xlim([xylim[0], xylim[1]])
    ax2.set_ylim([xylim[4], xylim[5]])
    ax2.spines['top'].set_color("none")
    ax2.spines['right'].set_color("none")

In [ ]:
def plot_distribution(gooddataRight, gooddataLeft, baddataRight, baddataLeft, barplotaxes, maxminstepbin, scatterplotaxes, xyLabels=["Speed cm/s","Quantity", "Direction", "Speed(cm/s)"], title=[None], linewidth = 1):      

    plt.figure(figsize = [15, 6])
    #rcParams['axes.linewidth'] = 1

    ax1 = plt.subplot(1, 3, 1)
    plt.hist(gooddataRight, np.arange(maxminstepbin[0],maxminstepbin[1], maxminstepbin[2]), weights=np.ones_like(gooddataRight)/float(len(gooddataRight + baddataRight)), color ='g')
    plt.hist(baddataRight, np.arange(maxminstepbin[0],maxminstepbin[1], maxminstepbin[2]), weights=np.ones_like(baddataRight)/float(len(gooddataRight + baddataRight)), color ='r', alpha = 0.2)
    ax1.set_title(title[0], fontsize = title[3])
    ax1.set_xlabel(xyLabels[0], fontsize = xyLabels[6])
    ax1.set_ylabel(xyLabels[1], fontsize = xyLabels[6])
    ax1.set_xlim([barplotaxes[0], barplotaxes[1]])
    ax1.set_ylim([barplotaxes[2], barplotaxes[3]])
    ax1.spines['bottom'].set_linewidth(linewidth[0])
    ax1.spines['left'].set_linewidth(linewidth[0])
    ax1.spines['top'].set_color("none")
    ax1.spines['right'].set_color("none")
    ax1.tick_params(width=2, labelsize = xyLabels[7])

    ax2 = plt.subplot(1, 3, 2)
    plt.hist(gooddataLeft, np.arange(maxminstepbin[0],maxminstepbin[1], maxminstepbin[2]), weights=np.ones_like(gooddataLeft)/float(len(gooddataLeft + baddataLeft)), color ='g')
    plt.hist(baddataLeft, np.arange(maxminstepbin[0],maxminstepbin[1], maxminstepbin[2]), weights=np.ones_like(baddataLeft)/float(len(gooddataLeft + baddataLeft)), color ='r', alpha = 0.2)
    ax2.set_title(title[1], fontsize = title[3])
    ax2.set_xlabel(xyLabels[0], fontsize = xyLabels[6])
    ax2.set_ylabel(xyLabels[1], fontsize = xyLabels[6])
    ax2.set_xlim([barplotaxes[0], barplotaxes[1]])
    ax2.set_ylim([barplotaxes[2], barplotaxes[3]])
    ax2.spines['bottom'].set_linewidth(linewidth[0])
    ax2.spines['left'].set_linewidth(linewidth[0])
    ax2.spines['top'].set_color("none")
    ax2.spines['right'].set_color("none")
    ax2.tick_params(width=2, labelsize = xyLabels[7])

    ax3 = plt.subplot(1, 3, 3)
    plt.scatter(np.random.normal(1, 0.05, len(gooddataRight)), gooddataRight, s=2, color = 'g')
    plt.scatter(np.random.normal(2, 0.05, len(gooddataLeft)), gooddataLeft, s=2, color = 'g')
    plt.scatter(np.random.normal(1, 0.05, len(baddataRight)), baddataRight, s=2, color = 'r', alpha = 0.2)
    plt.scatter(np.random.normal(2, 0.05, len(baddataLeft)), baddataLeft, s=2, color = 'r', alpha = 0.2)
    plt.scatter(1.2, np.mean(gooddataRight), s=25, color = 'g')
    plt.scatter(2.2, np.mean(gooddataLeft), s=25, color = 'g')
    plt.errorbar(1.2, np.mean(gooddataRight), yerr=scipy.stats.sem(gooddataRight))
    plt.errorbar(1.2, np.mean(gooddataLeft), yerr=scipy.stats.sem(gooddataLeft))
    plt.boxplot(gooddataRight, positions = [1.35])
    plt.boxplot(gooddataLeft, positions = [2.35])
    ax3.set_xlabel(xyLabels[2], fontsize = xyLabels[6])
    ax3.set_ylabel(xyLabels[3], fontsize = xyLabels[6])
    ax3.set_title(title[2], fontsize = title[3])
    ax3.set_xlim([scatterplotaxes[0], scatterplotaxes[1]])
    ax3.set_ylim([scatterplotaxes[2], scatterplotaxes[3]])
    plt.xticks([1,2], [xyLabels[4],xyLabels[5]], fontsize = xyLabels[6])
    ax3.spines['bottom'].set_linewidth(linewidth[0])
    ax3.spines['left'].set_linewidth(linewidth[0])
    ax3.spines['top'].set_color("none")
    ax3.spines['right'].set_color("none")
    ax3.tick_params(width=2, labelsize = xyLabels[7])

    plt.tight_layout(pad=0.5)
    return plt

In [ ]:
def plot_recap(animal, session, gooddataRight, gooddataLeft, baddataRight, baddataLeft, axes, xyLabels=["Speed cm/s","Quantity", "Direction", "Speed(cm/s)"], title=[None], linewidth = 1): 
    
    plt.figure(figsize = [6, 10])
    for animal in animalList:
        sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
        for session in sessionList:
            
            ax3 = plt.gca()
            plt.scatter(1, np.mean(gooddataRight[animal, session]), s=500, color =rat_markers[animal][0], marker=rat_markers[animal][1])
            plt.scatter(2, np.mean(gooddataLeft[animal, session]), s=500, color =rat_markers[animal][0], marker=rat_markers[animal][1])
            plt.plot((1, 2), (np.mean(gooddataRight[animal, session]),np.mean(gooddataLeft[animal, session])), color = rat_markers[animal][0], label=animal)
            plt.scatter(1, np.mean(list(itertools.chain.from_iterable(list(gooddataRight.values())))), s=40, color ='k')
            plt.scatter(2, np.mean(list(itertools.chain.from_iterable(list(gooddataLeft.values())))), s=40, color ='k')
            plt.plot((1, 2), (np.mean(list(itertools.chain.from_iterable(list(gooddataRight.values())))), 
                             np.mean(list(itertools.chain.from_iterable(list(gooddataLeft.values()))))), color='k')

            plt.scatter(1, np.mean(gooddataRight[animal, session] + baddataRight[animal, session]), s=500, color =rat_markers[animal][0], marker=rat_markers[animal][1], alpha = 0.05)
            plt.scatter(2, np.mean(gooddataLeft[animal, session] + baddataLeft[animal, session]), s=500, color =rat_markers[animal][0], marker=rat_markers[animal][1], alpha = 0.05)
            plt.plot((1, 2), (np.mean(gooddataRight[animal, session] + baddataRight[animal, session]), 
                               np.mean(gooddataLeft[animal, session] + baddataLeft[animal, session])), color = rat_markers[animal][0], label=animal, alpha = 0.05)
            plt.scatter(1, np.mean(list(itertools.chain.from_iterable(list(gooddataRight.values()))) + list(itertools.chain.from_iterable(list(baddataRight.values())))), s=40, color ='k', alpha = 0.01)
            plt.scatter(2, np.mean(list(itertools.chain.from_iterable(list(gooddataLeft.values()))) + list(itertools.chain.from_iterable(list(baddataLeft.values())))), s=40, color ='k', alpha = 0.01)
            plt.plot((1, 2), (np.mean(list(itertools.chain.from_iterable(list(gooddataRight.values()))) + list(itertools.chain.from_iterable(list(baddataRight.values())))), 
                              np.mean(list(itertools.chain.from_iterable(list(gooddataLeft.values()))) + list(itertools.chain.from_iterable(list(baddataLeft.values()))))), color='k', alpha = 0.01)

            ax3.set_xlabel(xyLabels[2], fontsize = xyLabels[4])
            ax3.set_ylabel(xyLabels[3], fontsize = xyLabels[4])
            ax3.set_title(title[0], fontsize = title[1])
            ax3.set_xlim([axes[0], axes[1]])
            ax3.set_ylim([axes[2], axes[3]])
            plt.xticks([1,2], [xyLabels[0],xyLabels[1]])
            ax3.spines['bottom'].set_linewidth(linewidth[0])
            ax3.spines['left'].set_linewidth(linewidth[0])
            ax3.spines['top'].set_color("none")
            ax3.spines['right'].set_color("none")
            ax3.tick_params(width=2, labelsize = xyLabels[5])
            plt.legend()
    plt.show()

# Extract Runs and Stays

In [ ]:
# dict to store the unfiltered runs to compute the limspeed for each rat and direction 
all_speedToRight = {}
all_speedToLeft = {}
# dict to store the computed limspeeds
limspeedToRight = {}
limspeedToLeft ={}

#arrays to store the speed in cm/s for the good and bad runs to the right and left that have been sorted with limspeed
good_speedToRight = {}
good_speedToLeft = {}
bad_speedToRight = {}
bad_speedToLeft = {}

#arrays to store the Xtrack of the run for plotting
good_x_trackToR = {}
good_x_trackToL = {}
bad_x_trackToR = {}
bad_x_trackToL = {}


all_timeInRight = {}
all_timeInLeft = {}
limstayRight = {}
limstayLeft ={}

good_timeInRight = {}
good_timeInLeft = {}
good_x_stayR = {}
good_x_stayL = {}

bad_timeInRight = {}
bad_timeInLeft = {}
bad_x_stayR = {}
bad_x_stayL = {}

for animal in animalList: 
        sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
        for session in sessionList:
            #extract all the runs to compute limspeed
            all_speedToRight[animal, session], all_speedToLeft[animal, session] = extract_runsRightLeftFiltered(rawPositionX, animal, session, boundary = params[animal, session]["boundaries"], limspeed = [None, None])[8:10]
            all_timeInRight[animal, session], all_timeInLeft[animal, session] = extract_runsRightLeftFiltered(rawPositionX, animal, session, boundary = params[animal, session]["boundaries"], limspeed = [None, None])[18:20]

            #compute limspeed (speed below which we consider that the rats are not running) as the speeds below 2 st dev of the mean speed. can be changed
            limspeedToRight[animal, session] = np.mean(all_speedToRight[animal, session]) - 2*statistics.stdev(all_speedToRight[animal, session]) #stdev of non normal sample ?
            limspeedToLeft[animal, session] = np.mean(all_speedToLeft[animal, session]) - 2*statistics.stdev(all_speedToLeft[animal, session])
            limstayRight[animal, session] = np.mean(all_timeInRight[animal, session]) + 2*statistics.stdev(all_timeInRight[animal, session])
            limstayLeft[animal, session] = np.mean(all_timeInLeft[animal, session]) + 2*statistics.stdev(all_timeInLeft[animal, session])

            #extract the run data (good/bad, to left/ to right) 
            good_speedToRight[animal, session], good_speedToLeft[animal, session], good_x_trackToR[animal, session], good_x_trackToL[animal, session], bad_speedToRight[animal, session], bad_speedToLeft[animal, session], bad_x_trackToR[animal, session], bad_x_trackToL[animal, session] = extract_runsRightLeftFiltered(rawPositionX, animal, session, boundary = params[animal, session]["boundaries"], limspeed = [limspeedToRight[animal, session], limspeedToLeft[animal, session]])[0:8]

            good_timeInRight[animal, session], good_timeInLeft[animal, session], good_x_stayR[animal, session], good_x_stayL[animal, session], bad_timeInRight[animal, session], bad_timeInLeft[animal, session], bad_x_stayR[animal, session], bad_x_stayL[animal, session] = extract_runsRightLeftFiltered(rawPositionX, animal, session, boundary = params[animal, session]["boundaries"], limspeed = [limstayRight[animal, session], limstayLeft[animal, session]])[10:18]
            print(animal, "done")

# COMPUTE STATISTICS

In [ ]:
print("**************RUNS***********")
print("*********NOT_FILTERED********")
#idea : do stats as statsrun(rightdata, leftdata, "", [rat1, rat2, rat3, Rat4], [etc]for groups or [Rat1], [rat2], [etc] for individuals)
#pool all data
all_speedToRightPooled = list(itertools.chain.from_iterable(list(all_speedToRight.values())))
all_speedToLeftPooled = list(itertools.chain.from_iterable(list(all_speedToLeft.values())))
good_speedToRightPooled = list(itertools.chain.from_iterable(list(good_speedToRight.values())))
good_speedToLeftPooled = list(itertools.chain.from_iterable(list(good_speedToLeft.values())))
bad_speedToRightPooled = list(itertools.chain.from_iterable(list(bad_speedToRight.values())))
bad_speedToLeftPooled = list(itertools.chain.from_iterable(list(bad_speedToLeft.values())))

for animal in animalList:
        sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
        for session in sessionList:   
            #compute stats for individual animals
            statsrun(all_speedToRight[animal, session], all_speedToLeft[animal, session], animal)
print()
print("************POOLED***********")
#compute stats for the group
statsrun(all_speedToRightPooled, all_speedToLeftPooled, "")
print()
print("*****************************")
print("***********FILTERED**********")
for animal in animalList:
        sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
        for session in sessionList:   
            statsrun(good_speedToRight[animal, session], good_speedToLeft[animal, session], animal)
print()
print("************POOLED***********")
statsrun(good_speedToRightPooled, good_speedToLeftPooled, "")

In [ ]:
print("*************STAYS***********")
print("*********NOT_FILTERED********")
#pool all data
all_timeInRightPooled = list(itertools.chain.from_iterable(list(all_timeInRight.values())))
all_timeInLeftPooled = list(itertools.chain.from_iterable(list(all_timeInLeft.values())))
good_timeInRightPooled = list(itertools.chain.from_iterable(list(good_timeInRight.values())))
good_timeInLeftPooled = list(itertools.chain.from_iterable(list(good_timeInLeft.values())))
bad_timeInRightPooled = list(itertools.chain.from_iterable(list(bad_timeInRight.values())))
bad_timeInLeftPooled = list(itertools.chain.from_iterable(list(bad_timeInLeft.values())))
for animal in animalList:
        sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
        for session in sessionList:   
            #compute stats for individual animals
            statsrun(all_timeInRight[animal, session], all_timeInLeft[animal, session], animal)
print()
print("************POOLED***********")
#compute stats for the group
statsrun(all_timeInRightPooled, all_timeInLeftPooled, "")
print()
print("*****************************")
print("***********FILTERED**********")
#idea : do stats as statsrun(rightdata, leftdata, "", [rat1, rat2, rat3, Rat4], [etc]for groups or [Rat1], [rat2], [etc] for individuals)
#pool all data

for animal in animalList:
        sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
        for session in sessionList:   
            #compute stats for individual animals
            statsrun(good_timeInRight[animal, session], good_timeInLeft[animal, session], animal)
print()
print("************POOLED***********")
#compute stats for the group
statsrun(good_timeInRightPooled, good_timeInLeftPooled, "")

# VISUALISE TRACKS

In [ ]:
for animal in animalList:
        sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
        for session in sessionList:
#Have the 4 plots on one line -> modify function 
            plot_tracks(animal, session, good_x_trackToR[animal, session], good_x_trackToL[animal, session], bad_x_trackToR[animal, session], bad_x_trackToL[animal, session], xylim = [-0.1, 2, 0/13.5, 1400/13.5, 0/13.5, 1400/13.5], xyLabels=["Run length (s)","X Position (cm)", 14, 12], title=["Run to Right", "Run to Left", 12], suptitle = [animal + " Run Tracks", 16], linewidth = [1.5])

In [ ]:
for animal in animalList:
        sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
        for session in sessionList:
#Have the 4 plots on one line -> modify function 
            plot_tracks(animal, session, good_x_stayR[animal, session], good_x_stayL[animal, session], bad_x_stayR[animal, session], bad_x_stayL[animal, session], xylim = [-2, 20, (params[animal, session]["boundaries"][0] - 5)/13.5, (params[animal, session]["boundaries"][0] + 130)/13.5, (params[animal, session]["boundaries"][1] - 130)/13.5, (params[animal, session]["boundaries"][1] + 5)/13.5], xyLabels=["Time in zone (s)","X Position (cm)", 14, 12], title=["Time in Right", "Time in Left", 12], suptitle = [animal + " Time in extremities", 16], linewidth = [1.5])
            plt.show()

# PLOT DISTRIBUTION 

In [ ]:
#IDEM, make one line with 6 plots
for animal in animalList:
        sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
        for session in sessionList:
            plot_distribution(good_speedToRight[animal, session], good_speedToLeft[animal, session], bad_speedToRight[animal, session], bad_speedToLeft[animal, session], barplotaxes = [0, 100, 0, 1], maxminstepbin = [0,100,10], scatterplotaxes = [0.5, 2.5, 0, 100], xyLabels=["Speed cm/s","%", "Direction of run", "Speed(cm/s)", "To Right", "To Left", 14, 12], title=[animal+"\n"+ "-->", animal+"\n"+ "<--", animal, 16], linewidth = [1.5])

plt.show()
print("########################################################################################################")
plot_distribution(good_speedToRightPooled, good_speedToLeftPooled, bad_speedToRightPooled, bad_speedToLeftPooled, barplotaxes = [0, 100, 0, 1], maxminstepbin = [0,100,10], scatterplotaxes = [0.5, 2.5, 0, 100], xyLabels=["Speed cm/s","%", "Direction of run", "Speed(cm/s)", "To Right", "To Left", 14, 12], title=["Pooled"+"\n"+ "-->", "Pooled"+"\n"+ "<--", "Pooled", 16], linewidth = [1.5])

In [ ]:
for animal in animalList:
        sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
        for session in sessionList:
            plot_distribution(good_timeInRight[animal, session], good_timeInLeft[animal, session], bad_timeInRight[animal, session], bad_timeInLeft[animal, session], barplotaxes = [0, 30, 0, 1], maxminstepbin = [0,30,1], scatterplotaxes = [0.5, 2.5, 0, 30], xyLabels=["Time in zone (s)","%", "Zone", "Time in zone (s)", "In Right", "In Left", 14, 12], title=[animal+"\n"+ "In Right", animal+"\n"+ "In Left", animal, 16], linewidth = [1.5])

plt.show()
print("########################################################################################################")
plot_distribution(good_timeInRightPooled, good_timeInLeftPooled, bad_timeInRightPooled, bad_timeInLeftPooled, barplotaxes = [0, 30, 0, 1], maxminstepbin = [0,30,1],scatterplotaxes = [0.5, 2.5, 0, 30], xyLabels=["Time in zone (s)","%", "Zone", "Time in zone (s)", "In Right", "In Left", 14, 12], title=["Pooled"+"\n"+ "In Right", "Pooled"+"\n"+ "In Left", "Pooled", 16], linewidth = [1.5])

# PLOT RECAPS

In [ ]:
plot_recap(animal, session, good_speedToRight, good_speedToLeft, bad_speedToRight, bad_speedToLeft, axes = [0.5, 2.5, 0, 100], xyLabels=["To Right", "To Left", "Direction", "Speed (cm/s)", 14, 12], title=["Recap runs", 16], linewidth = [1.5])

plot_recap(animal, session, good_timeInRight, good_timeInLeft, bad_timeInRight, bad_timeInLeft, axes = [0.5, 2.5, 0, 5], xyLabels=["In Right", "In Left", "Side", "Time: (s)", 14, 12], title=["Recap time in sides", 16], linewidth = [1.5])


# TimeBins

In [ ]:
timeBins = 6
for animal in animalList: 
        sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
        for session in sessionList:
            #create the N time bins
            binTime[animal, session] = bin_session(rawTime, rawTime, 0, acquisitionCutOff, timeBins)
            binPositionX[animal, session] = bin_session(rawPositionX, rawTime, 0, acquisitionCutOff, timeBins)

In [ ]:
def extract_runsRightLeftFilteredBin(data, animal, session, boundary, limspeed=None, timeBins = timeBins):   
    rawdata_run = {}
    rawdata_stay = {}
    rawdata_run[animal, session] = {}
    rawdata_stay[animal, session] = {}
    good_speedRight_cms = {}
    good_speedLeft_cms = {}
    good_x_trackR = {}
    good_x_trackL = {}

    bad_speedRight_cms = {}
    bad_speedLeft_cms = {}
    bad_x_trackR = {}
    bad_x_trackL = {}
    all_speedRight_cms = {} 
    all_speedLeft_cms = {}

    good_timeInRight = {}
    good_timeInLeft = {}
    good_x_stayR = {}
    good_x_stayL = {}

    bad_timeInRight = {}
    bad_timeInLeft = {}
    bad_x_stayR = {}
    bad_x_stayL = {}
    all_timeInRight = {}
    all_timeInLeft = {}


    for i in range(0, timeBins):
        rawdata_run[animal, session][i] = []
        rawdata_stay[animal, session][i] = []

        good_speedRight_cms[i] = []
        good_speedLeft_cms[i] = []
        good_x_trackR[i] = []
        good_x_trackL[i] = []

        bad_speedRight_cms[i] = []
        bad_speedLeft_cms[i] = []
        bad_x_trackR[i] = []
        bad_x_trackL[i] = []
        all_speedRight_cms[i] = []
        all_speedLeft_cms[i] = []


        good_timeInRight[i] = []
        good_timeInLeft[i] = []
        good_x_stayR[i] = []
        good_x_stayL[i] = []

        bad_timeInRight[i] = []
        bad_timeInLeft[i] = []
        bad_x_stayR[i] = []
        bad_x_stayL[i] = []
        all_timeInRight[i] = []
        all_timeInLeft[i] = []
        #crop the complete track at the set boundaries -> keep only the runs and set the rest to 0
        rawdata_run[animal, session][i] = [e/13.5 if e <= boundary[0] and e>= boundary[1] else 0 for e in data[animal, session][i]] #conversion pixel to cm maybe do this better 
        # use the split a list function to create a new sublist when there is a 0, which we just set up before
        for subsplit in split_a_list_at_zeros(rawdata_run[animal, session][i]):
            
            #check if the subsplit starts on the left or the right -> determine if the animal is running left or right
            if subsplit[0] < (((boundary[0]+boundary[1])/2) /13.5):
                #check if the subsplit is ending on the other side -> determine if this is a run 
                if subsplit[-1] > (((boundary[0]+boundary[1])/2) /13.5):
                    #calculate distance run as the distance between first and last value
                    distRight_cm = abs(subsplit[0]-subsplit[-1]) 
                    #calculate time as number of values (frames) * time per frame
                    timeRight = len(subsplit) * 0.05
                    #check if there is a limspeed : if no compute all the runs -> used to compute limspeed
                    #second pass : there is a computed limspeed, use it to sort good and bad runs
                    if limspeed[0] is not None:
                        #check if this is a good or bad run and assign im to the good or bad array accordingly
                        if distRight_cm/timeRight > limspeed[0]:
                            #good run array
                            good_speedRight_cms[i].append(distRight_cm/timeRight)
                            good_x_trackR[i].append(subsplit)     
                        else:
                            #bad run array
                            bad_speedRight_cms[i].append(distRight_cm/timeRight)
                            bad_x_trackR[i].append(subsplit)
                    else:
                        #all runs array first pass
                        all_speedRight_cms[i].append(distRight_cm/timeRight)

            #same thing for the runs that go to the other side
            elif subsplit[0]>(((boundary[0]+boundary[1])/2)/13.5):
                if subsplit[-1]<(((boundary[0]+boundary[1])/2)/13.5):
                    distLeft_cm = abs(subsplit[0]-subsplit[-1])
                    timeLeft = len(subsplit) * 0.05
                    if limspeed[1] is not None:
                        if distLeft_cm/timeLeft > limspeed[1]:
                            good_speedLeft_cms[i].append(distLeft_cm/timeLeft)   
                            good_x_trackL[i].append(subsplit)
                        else:
                            bad_speedLeft_cms[i].append(distLeft_cm/timeLeft)
                            bad_x_trackL[i].append(subsplit)
                    else:
                        all_speedLeft_cms[i].append(distLeft_cm/timeLeft)

        # inverse crop, at the boundaries keep the distal parts, same process as above
        rawdata_stay[animal, session][i] = [e/13.5 if e >= boundary[0] or e<= boundary[1] else 0 for e in data[animal, session][i]]
        for subsplit in split_a_list_at_zeros(rawdata_stay[animal, session][i]):
            #first identify if the subsplit created is on the left or right by comparing to the middle
            if subsplit[0] > (((boundary[0]+boundary[1])/2) /13.5):
                # maybe add another filter i.e. out if less than 5 frames
                #same idea as above
                if limspeed[0] is not None:
                    #add lenght (frames) of the subsplit * time per frame
                    if len(subsplit)* 0.05 < limspeed[0]:
                        good_timeInRight[i].append(len(subsplit)* 0.05)#clean 
                        good_x_stayR[i].append(subsplit)
                    else:
                        bad_timeInRight[i].append(len(subsplit)* 0.05)
                        bad_x_stayR[i].append(subsplit)
                else:
                    all_timeInRight[i].append(len(subsplit)* 0.05)
                    
            elif subsplit[0]<(((boundary[0]+boundary[1])/2)/13.5):
                #compute
                if limspeed[1] is not None:
                    if len(subsplit)* 0.05 < limspeed[1]:
                        good_timeInLeft[i].append(len(subsplit)* 0.05)   
                        good_x_stayL[i].append(subsplit)
                    else:
                        bad_timeInLeft[i].append(len(subsplit)* 0.05)
                        bad_x_stayL[i].append(subsplit)
                else:
                    all_timeInLeft[i].append(len(subsplit)* 0.05)

    return good_speedRight_cms, good_speedLeft_cms, good_x_trackR, good_x_trackL, bad_speedRight_cms, bad_speedLeft_cms, bad_x_trackR, bad_x_trackL, all_speedRight_cms, all_speedLeft_cms, good_timeInRight, good_timeInLeft, good_x_stayR, good_x_stayL, bad_timeInRight, bad_timeInLeft, bad_x_stayR, bad_x_stayL, all_timeInRight, all_timeInLeft

In [ ]:
# dict to store the unfiltered runs to compute the limspeed for each rat and direction 
all_speedToRightBin = {}
all_speedToLeftBin = {}
# dict to store the computed limspeeds
limspeedToRightBin = {}
limspeedToLeftBin ={}

#arrays to store the speed in cm/s for the good and bad runs to the right and left that have been sorted with limspeed
good_speedToRightBin = {}
good_speedToLeftBin = {}
bad_speedToRightBin = {}
bad_speedToLeftBin = {}

#arrays to store the Xtrack of the run for plotting
good_x_trackToRBin = {}
good_x_trackToLBin = {}
bad_x_trackToRBin = {}
bad_x_trackToLBin = {}


all_timeInRightBin = {}
all_timeInLeftBin = {}
limstayRightBin = {}
limstayLeftBin ={}

good_timeInRightBin = {}
good_timeInLeftBin = {}
good_x_stayRBin = {}
good_x_stayLBin = {}

bad_timeInRightBin = {}
bad_timeInLeftBin = {}
bad_x_stayRBin = {}
bad_x_stayLBin = {}

for animal in animalList: 
        sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
        for session in sessionList:
            #extract all the runs to compute limspeed
            all_speedToRightBin[animal, session], all_speedToLeftBin[animal, session] = extract_runsRightLeftFilteredBin(binPositionX, animal, session, boundary = params[animal, session]["boundaries"], limspeed = [None, None], timeBins = timeBins)[8:10]
            all_timeInRightBin[animal, session], all_timeInLeftBin[animal, session] = extract_runsRightLeftFilteredBin(binPositionX, animal, session, boundary = params[animal, session]["boundaries"], limspeed = [None, None], timeBins = timeBins)[18:20]

            #compute limspeed (speed below which we consider that the rats are not running) as the speeds below 2 st dev of the mean speed. can be changed
            limspeedToRightBin[animal, session] =  limspeedToRight[animal, session]
            limspeedToLeftBin[animal, session] =limspeedToLeft[animal, session]
            limstayRightBin[animal, session] = limstayRight[animal, session]
            limstayLeftBin[animal, session] = limstayLeft[animal, session]

            #extract the run data (good/bad, to left/ to right) 
            good_speedToRightBin[animal, session], good_speedToLeftBin[animal, session], good_x_trackToRBin[animal, session], good_x_trackToLBin[animal, session], bad_speedToRightBin[animal, session], bad_speedToLeftBin[animal, session], bad_x_trackToRBin[animal, session], bad_x_trackToLBin[animal, session] = extract_runsRightLeftFilteredBin(binPositionX, animal, session, boundary = params[animal, session]["boundaries"], limspeed = [limspeedToRight[animal, session], limspeedToLeft[animal, session]], timeBins = timeBins)[0:8]

            good_timeInRightBin[animal, session], good_timeInLeftBin[animal, session], good_x_stayRBin[animal, session], good_x_stayLBin[animal, session], bad_timeInRightBin[animal, session], bad_timeInLeftBin[animal, session], bad_x_stayRBin[animal, session], bad_x_stayLBin[animal, session] = extract_runsRightLeftFilteredBin(binPositionX, animal, session, boundary = params[animal, session]["boundaries"], limspeed = [limstayRight[animal, session], limstayLeft[animal, session]], timeBins = timeBins)[10:18]
            print(animal, "done")





all_speedToRightPooledBin = {}
all_speedToLeftPooledBin = {}
good_speedToRightPooledBin = {}
good_speedToLeftPooledBin = {}
bad_speedToRightPooledBin = {}
bad_speedToLeftPooledBin = {}
good_x_trackToRPooledBin = {}
good_x_trackToLPooledBin = {}
bad_x_trackToRPooledBin = {}
bad_x_trackToLPooledBin = {}


all_timeInRightPooledBin = {}
all_timeInLeftPooledBin = {}
good_timeInRightPooledBin = {}
good_timeInLeftPooledBin = {}
good_x_stayRPooledBin = {}
good_x_stayLPooledBin = {}
bad_timeInRightPooledBin = {}
bad_timeInLeftPooledBin = {}
bad_x_stayRPooledBin = {}
bad_x_stayLPooledBin = {}

for i in range(0, timeBins):

    all_speedToRightPooledBin[i] = []
    all_speedToLeftPooledBin[i] = []
    good_speedToRightPooledBin[i] = []
    good_speedToLeftPooledBin[i] = []
    bad_speedToRightPooledBin[i] = []
    bad_speedToLeftPooledBin[i] = []
    good_x_trackToRPooledBin[i] = []
    good_x_trackToLPooledBin[i] = []
    bad_x_trackToRPooledBin[i] = []
    bad_x_trackToLPooledBin[i] = []


    all_timeInRightPooledBin[i] = []
    all_timeInLeftPooledBin[i] = []
    good_timeInRightPooledBin[i] = []
    good_timeInLeftPooledBin[i] = []
    good_x_stayRPooledBin[i] = []
    good_x_stayLPooledBin[i] = []
    bad_timeInRightPooledBin[i] = []
    bad_timeInLeftPooledBin[i] = []
    bad_x_stayRPooledBin[i] = []
    bad_x_stayLPooledBin[i] = []

    for animal in animalList:
        sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
        for session in sessionList:

            all_speedToRightPooledBin[i].extend(all_speedToRightBin[animal, session][i])
            all_speedToLeftPooledBin[i].extend(all_speedToLeftBin[animal, session][i])
            good_speedToRightPooledBin[i].extend(good_speedToRightBin[animal, session][i])
            good_speedToLeftPooledBin[i].extend(good_speedToLeftBin[animal, session][i])
            bad_speedToRightPooledBin[i].extend(bad_speedToRightBin[animal, session][i])
            bad_speedToLeftPooledBin[i].extend(bad_speedToLeftBin[animal, session][i])
            good_x_trackToRPooledBin[i].extend(good_x_trackToRBin[animal, session][i])
            good_x_trackToLPooledBin[i].extend(good_x_trackToLBin[animal, session][i])
            bad_x_trackToRPooledBin[i].extend(bad_x_trackToRBin[animal, session][i])
            bad_x_trackToLPooledBin[i].extend(bad_x_trackToLBin[animal, session][i])


            all_timeInRightPooledBin[i].extend(all_timeInRightBin[animal, session][i])
            all_timeInLeftPooledBin[i].extend(all_timeInLeftBin[animal, session][i])
            good_timeInRightPooledBin[i].extend(good_timeInRightBin[animal, session][i])
            good_timeInLeftPooledBin[i].extend(good_timeInLeftBin[animal, session][i])
            good_x_stayRPooledBin[i].extend(good_x_stayRBin[animal, session][i])
            good_x_stayLPooledBin[i].extend(good_x_stayLBin[animal, session][i])
            bad_timeInRightPooledBin[i].extend(bad_timeInRightBin[animal, session][i])
            bad_timeInLeftPooledBin[i].extend(bad_timeInLeftBin[animal, session][i])
            bad_x_stayRPooledBin[i].extend(bad_x_stayRBin[animal, session][i])
            bad_x_stayLPooledBin[i].extend(bad_x_stayLBin[animal, session][i])

# stats

In [ ]:
def plot_tracks2(animal, session, gooddataRight, gooddataLeft, baddataRight, baddataLeft, xylim, xyLabels=["Speed cm/s","Quantity", "Direction", "Speed(cm/s)"], title=[None], suptitle = [""], linewidth = 1): 
    plt.figure(figsize = [5*timeBins, 5])
    for m in range(0, timeBins):
            
        
        plt.suptitle(suptitle[0], fontsize = suptitle[1])
        ax = plt.subplot(2, timeBins, m+1)
        for i in gooddataRight[m]:
            plt.plot(np.linspace(0, len(i)*0.05, len(i)), i, color='g' , linewidth=0.3)
        for k in baddataRight[m]:
            plt.plot(np.linspace(0, len(k)*0.05, len(k)), k, color='r', linewidth=0.3)

        ax.set_title(title[0], fontsize = title[2])
        ax.set_xlabel(xyLabels[0], fontsize = xyLabels[3])
        ax.set_ylabel(xyLabels[1], fontsize = xyLabels[3])
        ax.set_xlim([xylim[0], xylim[1]])
        ax.set_ylim([xylim[2], xylim[3]])
        ax.spines['top'].set_color("none")
        ax.spines['right'].set_color("none")

    plt.figure(figsize = [5*timeBins, 5])
    for n in range(0, timeBins):   
        ax2 = plt.subplot(2, timeBins, n+1)
        for j in gooddataLeft[n]:
            plt.plot(np.linspace(0, len(j)*0.05, len(j)), j, color='g', linewidth=0.3)
        for l in baddataLeft[n]:
            plt.plot(np.linspace(0, len(l)*0.05, len(l)), l, color='r', linewidth=0.3)

        ax2.set_title(title[1], fontsize = title[2])
        ax2.set_xlabel(xyLabels[0], fontsize = xyLabels[3])
        ax2.set_ylabel(xyLabels[1], fontsize = xyLabels[3])
        ax2.set_xlim([xylim[0], xylim[1]])
        ax2.set_ylim([xylim[4], xylim[5]])
        ax2.spines['top'].set_color("none")
        ax2.spines['right'].set_color("none")

In [ ]:
for animal in animalList:
        sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
        for session in sessionList:
#Have the 4 plots on one line -> modify function 
            plot_tracks2(animal, session, good_x_trackToRBin[animal, session], good_x_trackToLBin[animal, session], bad_x_trackToRBin[animal, session], bad_x_trackToLBin[animal, session], xylim = [-0.1, 2, 0/13.5, 1400/13.5, 0/13.5, 1400/13.5], xyLabels=["Run length (s)","X Position (cm)", 14, 12], title=["Run to Right", "Run to Left", 12], suptitle = [animal + " Run Tracks", 16], linewidth = [1.5])

In [ ]:
for animal in animalList:
        sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
        for session in sessionList:
#Have the 4 plots on one line -> modify function 
            plot_tracks2(animal, session, good_x_stayRBin[animal, session], good_x_stayLBin[animal, session], bad_x_stayRBin[animal, session], bad_x_stayLBin[animal, session], xylim = [-2, 20, (params[animal, session]["boundaries"][0] - 5)/13.5, (params[animal, session]["boundaries"][0] + 130)/13.5, (params[animal, session]["boundaries"][1] - 130)/13.5, (params[animal, session]["boundaries"][1] + 5)/13.5], xyLabels=["Time in zone (s)","X Position (cm)", 14, 12], title=["Time in Right", "Time in Left", 12], suptitle = [animal + " Time in extremities", 16], linewidth = [1.5])
            plt.show()

In [ ]:
  def plot_runs2(gooddataRight, gooddataLeft, baddataRight, baddataLeft, barplotaxes, xyLabels=["Speed cm/s","Quantity", "Direction", "Speed(cm/s)"], title=[None], linewidth = 1):

    for i in range(0, timeBins):

        ax1 = plt.subplot(1,2,1)
        plt.scatter(np.full((1, 1), i+1), np.mean(gooddataRight[i]), s=500, marker=rat_markers[animal][1], color = rat_markers[animal][0],label = animal if i == 0 else "")
        for i in range(0, timeBins-1):
            plt.plot(([i+1], [i+2]), (np.mean(gooddataRight[i]), np.mean(gooddataRight[i+1])), color = rat_markers[animal][0], linewidth = 0.5)

        ax1.set_title(title[0], fontsize = title[3])
        ax1.set_xlabel(xyLabels[0], fontsize = xyLabels[6])
        ax1.set_ylabel(xyLabels[1], fontsize = xyLabels[6])
        ax1.set_xlim([barplotaxes[0], barplotaxes[1]])
        ax1.set_ylim([barplotaxes[2], barplotaxes[3]])
        ax1.spines['bottom'].set_linewidth(linewidth[0])
        ax1.spines['left'].set_linewidth(linewidth[0])
        ax1.spines['top'].set_color("none")
        ax1.spines['right'].set_color("none")
        ax1.tick_params(width=2, labelsize = xyLabels[7])
        plt.legend()

    for i in range(0, timeBins):

        ax3 = plt.subplot(1,2,2)
        plt.scatter(np.full((1, 1), i+1), np.mean(gooddataLeft[i]), s=500, marker=rat_markers[animal][1], color = rat_markers[animal][0])
        for i in range(0, timeBins-1):
            plt.plot(([i+1], [i+2]), (np.mean(gooddataLeft[i]), np.mean(gooddataLeft[i+1])), color = rat_markers[animal][0], linewidth = 0.5)

        ax3.set_title(title[1], fontsize = title[3])
        ax3.set_xlabel(xyLabels[0], fontsize = xyLabels[6])
        ax3.set_ylabel(xyLabels[1], fontsize = xyLabels[6])
        ax3.set_xlim([barplotaxes[0], barplotaxes[1]])
        ax3.set_ylim([barplotaxes[2], barplotaxes[3]])
        ax3.spines['bottom'].set_linewidth(linewidth[0])
        ax3.spines['left'].set_linewidth(linewidth[0])
        ax3.spines['top'].set_color("none")
        ax3.spines['right'].set_color("none")
        ax3.tick_params(width=2, labelsize = xyLabels[7])
        #plt.legend()
    plt.tight_layout(pad=1.5)
    return plt

In [ ]:
plt.figure(figsize = [10, 6])
for animal in animalList:
        sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
        for session in sessionList:
           
            plot_runs2(good_speedToRightBin[animal, session], good_speedToLeftBin[animal, session], bad_speedToRightBin[animal, session], bad_speedToLeftBin[animal, session], barplotaxes = [0.5, 6.5, 0, 100], xyLabels=["Time Bin","Speed(cm/s)", "Time Bin", "Speed(cm/s)", "To Right", "To Left", 14, 12], title=["All Rats"+"\n"+ "-->", "All Rats"+"\n"+ "<--", "", 16], linewidth = [1.5])
plt.show()  

plt.figure(figsize = [10, 6])
for animal in animalList:
        sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
        for session in sessionList:
            plot_runs2(good_timeInRightBin[animal, session], good_timeInLeftBin[animal, session], bad_timeInRightBin[animal, session], bad_timeInLeftBin[animal, session], barplotaxes = [0.5, 6.5, 0, 5], xyLabels=["Time Bin","Speed(cm/s)", "Time Bin", "Speed(cm/s)", "To Right", "To Left", 14, 12], title=["All Rats"+"\n"+ "In Right", "All Rats"+"\n"+ "In Left", "", 16], linewidth = [1.5])

plt.show()

In [ ]:
def plot_runs3(gooddataRight, gooddataLeft, baddataRight, baddataLeft, barplotaxes, xyLabels=["Speed cm/s","Quantity", "Direction", "Speed(cm/s)"], title=[None], linewidth = 1):

    for i in range(0, timeBins):
        ax2 = plt.gca()
        plt.scatter(np.full((1, 1), i+1), np.mean(gooddataRight[i]), s=25, color = 'c', label = "Right" if i == 0 else "")
        plt.scatter(np.full((1, 1), i+1), np.mean(gooddataLeft[i]), s=25, color = 'b', label = "Left" if i == 0 else "")
        for i in range(0, timeBins-1):
            plt.plot(([i+1], [i+2]), (np.mean(gooddataRight[i]), np.mean(gooddataRight[i+1])), color = 'c', linewidth = 0.5)
        for i in range(0, timeBins-1):
            plt.plot(([i+1], [i+2]), (np.mean(gooddataLeft[i]), np.mean(gooddataLeft[i+1])), color = 'b', linewidth = 0.5)

        ax2.set_title(title[0], fontsize = title[3])
        ax2.set_xlabel(xyLabels[0], fontsize = xyLabels[6])
        ax2.set_ylabel(xyLabels[1], fontsize = xyLabels[6])
        ax2.set_xlim([barplotaxes[0], barplotaxes[1]])
        ax2.set_ylim([barplotaxes[2], barplotaxes[3]])
        ax2.spines['bottom'].set_linewidth(linewidth[0])
        ax2.spines['left'].set_linewidth(linewidth[0])
        ax2.spines['top'].set_color("none")
        ax2.spines['right'].set_color("none")
        ax2.tick_params(width=2, labelsize = xyLabels[7])
        plt.legend()
        
    plt.tight_layout(pad=0.5)
    return plt

In [ ]:
#add alpha in params to merge the graphs with the bad runs 
plot_runs3(good_speedToRightPooledBin, good_speedToLeftPooledBin, bad_speedToRightBin[animal, session], bad_speedToLeftBin[animal, session], barplotaxes = [0.5, 6.5, 0, 100], xyLabels=["Time Bin","Speed(cm/s)", "Time Bin", "Speed(cm/s)", "", "", 14, 12], title=["All Rats Pooled"+"\n"+ "To Left vs. To Right", "All Rats Pooled"+"\n"+ "In Left vs. In Right", "", 16], linewidth = [1.5])
plt.show()
plot_runs3(good_timeInRightPooledBin, good_timeInLeftPooledBin, bad_timeInRightBin[animal, session], bad_timeInLeftBin[animal, session], barplotaxes = [0.5, 6.5, 0, 5], xyLabels=["Time Bin","Time (s)", "Time Bin", "Speed(cm/s)", "", "", 14, 12], title=["All Rats Pooled"+"\n"+ "In Left vs. In Right", "All Rats Pooled"+"\n"+ "In Left vs. In Right", "", 16], linewidth = [1.5])
plt.show()

In [ ]:
plot_runs3(good_speedToRightPooledBin, good_speedToLeftPooledBin, bad_speedToRightBin[animal, session], bad_speedToLeftBin[animal, session], barplotaxes = [0.5, 6.5, 0, 100], xyLabels=["Time Bin","Speed(cm/s)", "Time Bin", "Speed(cm/s)", "", "", 14, 12], title=["All Rats Pooled"+"\n"+ "To Left vs. To Right", "All Rats Pooled"+"\n"+ "In Left vs. In Right", "", 16], linewidth = [1.5])
plot_runs3(all_speedToRightPooledBin, all_speedToLeftPooledBin, bad_speedToRightBin[animal, session], bad_speedToLeftBin[animal, session], barplotaxes = [0.5, 6.5, 0, 100], xyLabels=["Time Bin","Speed(cm/s)", "Time Bin", "Speed(cm/s)", "", "", 14, 12], title=["All Rats Pooled"+"\n"+ "To Left vs. To Right", "All Rats Pooled"+"\n"+ "In Left vs. In Right", "", 16], linewidth = [1.5])
plt.show()
plot_runs3(good_timeInRightPooledBin, good_timeInLeftPooledBin, bad_timeInRightBin[animal, session], bad_timeInLeftBin[animal, session], barplotaxes = [0.5, 6.5, 0, 5], xyLabels=["Time Bin","Speed(cm/s)", "Time Bin", "Speed(cm/s)", "", "", 14, 12], title=["All Rats Pooled"+"\n"+ "To Left vs. To Right", "All Rats Pooled"+"\n"+ "In Left vs. In Right", "", 16], linewidth = [1.5])
plot_runs3(all_timeInRightPooledBin, all_timeInLeftPooledBin, bad_timeInRightBin[animal, session], bad_timeInLeftBin[animal, session], barplotaxes = [0.5, 6.5, 0, 5], xyLabels=["Time Bin","Speed(cm/s)", "Time Bin", "Speed(cm/s)", "", "", 14, 12], title=["All Rats Pooled"+"\n"+ "To Left vs. To Right", "All Rats Pooled"+"\n"+ "In Left vs. In Right", "", 16], linewidth = [1.5])
plt.show()

# Number of runs per 5 min 

In [ ]:
nb_trials_to_right = {}
nb_trials_to_left = {}
nb_trials = {}

for animal in animalList:
    sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
    for session in sessionList:
        nb_trials_to_right[animal, session] = {}
        nb_trials_to_left[animal, session] = {}  
        nb_trials[animal, session] = {}  
        for i in range (0, timeBins):
            nb_trials_to_right[animal, session][i] = []
            nb_trials_to_left[animal, session][i] = []
            nb_trials[animal, session][i] = []

            nb_trials_to_right[animal, session][i] = len(good_speedToRightBin[animal, session][i])
            nb_trials_to_left[animal, session][i] = len(good_speedToLeftBin[animal, session][i])
            nb_trials[animal, session][i] = len(good_speedToRightBin[animal, session][i]) + len(good_speedToLeftBin[animal, session][i])
            #print(animal, i,  "ToRight", len(good_speedToRightBin[animal, session][i]), "ToLeft", len(good_speedToLeftBin[animal, session][i]))
        #print()

poolR = {}
poolL = {}
pool = {}
for i in range(0, timeBins):
    poolR[i] = []
    poolL[i] = []
    pool[i] = []
    for animal in animalList:
        sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
        for session in sessionList:
            poolR[i].append(nb_trials_to_right[animal, session][i])
            poolL[i].append(nb_trials_to_left[animal, session][i])
            pool[i].append(nb_trials_to_right[animal, session][i]+nb_trials_to_left[animal, session][i])

In [ ]:
def plot_runs4(gooddataRight, gooddataLeft, baddataRight, baddataLeft, barplotaxes, xyLabels=["Speed cm/s","Quantity", "Direction", "Speed(cm/s)"], title=[None], linewidth = 1):

    for i in range(0, timeBins):
        ax2 = plt.gca()
        plt.scatter(np.full((1, 1), i+1), gooddataRight[animal, session][i], s=500, marker=rat_markers[animal][1], color = rat_markers[animal][0],label = animal if i == 0 else "")
        plt.scatter(np.full((1, 1), i+1), gooddataLeft[animal, session][i], s=500, marker=rat_markers[animal][1], color = rat_markers[animal][0])
        for i in range(0, timeBins-1):
            plt.plot(([i+1], [i+2]), (gooddataRight[animal, session][i], gooddataRight[animal, session][i+1]), color = rat_markers[animal][0], linewidth = 1, linestyle = '-.')
        for i in range(0, timeBins-1):
            plt.plot(([i+1], [i+2]), (gooddataLeft[animal, session][i], gooddataLeft[animal, session][i+1]), color = rat_markers[animal][0], linewidth = 1)

        ax2.set_title(title[0], fontsize = title[3])
        ax2.set_xlabel(xyLabels[0], fontsize = xyLabels[6])
        ax2.set_ylabel(xyLabels[1], fontsize = xyLabels[6])
        ax2.set_xlim([barplotaxes[0], barplotaxes[1]])
        ax2.set_ylim([barplotaxes[2], barplotaxes[3]])
        ax2.spines['bottom'].set_linewidth(linewidth[0])
        ax2.spines['left'].set_linewidth(linewidth[0])
        ax2.spines['top'].set_color("none")
        ax2.spines['right'].set_color("none")
        ax2.tick_params(width=2, labelsize = xyLabels[7])
        plt.legend()

    plt.tight_layout(pad=0.5)
    return plt


In [ ]:
plt.figure(figsize = [10, 10])
for animal in animalList:
        sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
        for session in sessionList:
            plot_runs4(nb_trials_to_right, nb_trials_to_left, bad_timeInRightBin[animal, session], bad_timeInLeftBin[animal, session], barplotaxes = [0.5, 6.5, 0, 100], xyLabels=["Time Bin","Speed(cm/s)", "Time Bin", "Speed(cm/s)", "", "", 14, 12], title=["All Rats Pooled"+"\n"+ "To Left vs. To Right", "All Rats Pooled"+"\n"+ "In Left vs. In Right", "", 16], linewidth = [1.5])
plt.show()

In [ ]:
def plot_runs5(gooddataRight, gooddataLeft, baddataRight, baddataLeft, barplotaxes, xyLabels=["Speed cm/s","Quantity", "Direction", "Speed(cm/s)"], title=[None], linewidth = 1):
    for i in range(0, timeBins):

        ax2 = plt.gca()
        plt.scatter(np.full((1, 1), i+1), gooddataRight[animal, session][i], s=500, marker=rat_markers[animal][1], color = rat_markers[animal][0],label = animal if i == 0 else "")
        plt.scatter(np.full((1, 1), i+1), np.mean(gooddataLeft[i]), s=100, color = 'k', label = "Average" if i == 0 and animal == "RatM03" else "")
        for i in range(0, timeBins-1):
            plt.plot(([i+1], [i+2]), (gooddataRight[animal, session][i], gooddataRight[animal, session][i+1]), color = rat_markers[animal][0], linewidth = 1)
            plt.plot(([i+1], [i+2]), (np.mean(gooddataLeft[i]), np.mean(gooddataLeft[i+1])), color = 'k', linewidth = 1)
        
        ax2.set_title(title[0], fontsize = title[3])
        ax2.set_xlabel(xyLabels[0], fontsize = xyLabels[6])
        ax2.set_ylabel(xyLabels[1], fontsize = xyLabels[6])
        ax2.set_xlim([barplotaxes[0], barplotaxes[1]])
        ax2.set_ylim([barplotaxes[2], barplotaxes[3]])
        ax2.spines['bottom'].set_linewidth(linewidth[0])
        ax2.spines['left'].set_linewidth(linewidth[0])
        ax2.spines['top'].set_color("none")
        ax2.spines['right'].set_color("none")
        ax2.tick_params(width=2, labelsize = xyLabels[7])
        plt.legend()

    plt.tight_layout(pad=0.5)
    return plt

In [ ]:
plt.figure(figsize = [10, 10])
for animal in animalList:
        sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
        for session in sessionList:
            plot_runs5(nb_trials, pool, bad_timeInRightBin[animal, session], bad_timeInLeftBin[animal, session], barplotaxes = [0.5, 6.5, 0, 200], xyLabels=["Time Bin","Speed(cm/s)", "Time Bin", "nb of trials left + right", "", "", 14, 12], title=["All Rats"+"\n"+ "NB RUNS To Left + NB RUNS To Right", "All Rats"+"\n"+ "In Left vs. In Right", "", 16], linewidth = [1.5])
plt.show()

# test donut

In [ ]:
plt.figure(figsize = [20, 5])
for animal in animalList:
    sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
    for session in sessionList: 
        plt.figure(figsize = [30, 5])
        print(animal, session)
        plt.plot(rawPositionX[animal, session],rawPositionY[animal, session],[0,1400,1],[0,150,1], linewidth=0.1)
        plt.plot(rawLickLeft1X[animal, session],rawPositionY[animal, session],[0,1400,1],[0,150,1], linewidth=0.1, marker='o', markersize=0.1)
        plt.plot(rawLickRight1X[animal, session],rawPositionY[animal, session],[0,1400,1],[0,150,1], linewidth=0.1, marker='o', markersize=0.1)

# weight

In [ ]:
plt.figure(figsize = [6, 8])
for animal in animalList:
        sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
        for session in sessionList:
            
            xline = [params[animal, session]["weight"], params[animal, session]["weight"]]
            yline = [np.mean(good_speedToRight[animal, session]),np.mean(good_speedToLeft[animal, session])]
            
            plt.xlabel('Rat Weight (g)')
            plt.ylabel('speed(cm/s)')
            plt.title('Effect of weight on speed')

            plt.scatter(params[animal, session]["weight"], np.mean(good_speedToRight[animal, session]), s=25, color =rat_markers[animal][0], marker=rat_markers[animal][1], label = "toRight")
            plt.scatter(params[animal, session]["weight"], np.mean(good_speedToLeft[animal, session]), s=25, color =rat_markers[animal][0], marker=rat_markers[animal][1], label = "toLeft")
            plt.plot(xline, yline, ":", color='k')
            plt.axis([100, 500, 20, 80 ])
            plt.legend()

In [ ]:
for animal in animalList: 
    sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
    for session in sessionList: 
        plt.figure(figsize = [20, 5])
        print(animal, session)
        for i in range(2,4):
            plot_BASEtrajectory(binTime[animal, session][i],binPositionX[animal, session][i], np.zeros(len(binPositionX[animal, session][i])), np.zeros(len(binPositionX[animal, session][i])), [0,1800,1],[0,1400,1],  color = ["b", "c"], marker = ["", "o", 1], linewidth = [0.5], xyLabels=["Position (cm)", "Time(s)"])

                            #plot_BASEtrajectory(rawTime[animal, session],rawPositionX[animal, session], rawLickLeft1X[animal, session], rawLickRight1X[animal, session], [0,1800,1],[0,1400,1],  color = ["b", "c"], marker = ["", "o", 1], linewidth = [0.5], xyLabels=["Position (cm)", "Time(s)"])

# test cumul plot

In [ ]:
def plot_distribution(gooddataRight, gooddataLeft, baddataRight, baddataLeft, barplotaxes, maxminstepbin, scatterplotaxes, xyLabels=["Speed cm/s","Quantity", "Direction", "Speed(cm/s)"], title=[None], linewidth = 1):      

    plt.figure(figsize = [15, 6])
    #rcParams['axes.linewidth'] = 1

    ax1 = plt.subplot(1, 3, 1)
    plt.hist(gooddataRight, np.arange(maxminstepbin[0],maxminstepbin[1], maxminstepbin[2]), weights=np.ones_like(gooddataRight)/float(len(gooddataRight + baddataRight)), color ='g', histtype = 'step', cumulative = True, linewidth = 1)
    plt.hist(gooddataLeft, np.arange(maxminstepbin[0],maxminstepbin[1], maxminstepbin[2]), weights=np.ones_like(gooddataLeft)/float(len(gooddataLeft + baddataLeft)), color ='g', histtype = 'step', cumulative = True, linewidth = 5)
    #plt.hist(baddataRight, np.arange(maxminstepbin[0],maxminstepbin[1], maxminstepbin[2]), weights=np.ones_like(baddataRight)/float(len(gooddataRight + baddataRight)), color ='r', alpha = 0.2)
    ax1.set_title(title[0], fontsize = title[3])
    ax1.set_xlabel(xyLabels[0], fontsize = xyLabels[6])
    ax1.set_ylabel(xyLabels[1], fontsize = xyLabels[6])
    ax1.set_xlim([barplotaxes[0], barplotaxes[1]])
    ax1.set_ylim([barplotaxes[2], barplotaxes[3]])
    ax1.spines['bottom'].set_linewidth(linewidth[0])
    ax1.spines['left'].set_linewidth(linewidth[0])
    ax1.spines['top'].set_color("none")
    ax1.spines['right'].set_color("none")
    ax1.tick_params(width=2, labelsize = xyLabels[7])

    ax2 = plt.subplot(1, 3, 2)
    #plt.hist(gooddataLeft, np.arange(maxminstepbin[0],maxminstepbin[1], maxminstepbin[2]), weights=np.ones_like(gooddataLeft)/float(len(gooddataLeft + baddataLeft)), color ='g')
    #plt.hist(baddataLeft, np.arange(maxminstepbin[0],maxminstepbin[1], maxminstepbin[2]), weights=np.ones_like(baddataLeft)/float(len(gooddataLeft + baddataLeft)), color ='r', alpha = 0.2)
    ax2.set_title(title[1], fontsize = title[3])
    ax2.set_xlabel(xyLabels[0], fontsize = xyLabels[6])
    ax2.set_ylabel(xyLabels[1], fontsize = xyLabels[6])
    ax2.set_xlim([barplotaxes[0], barplotaxes[1]])
    ax2.set_ylim([barplotaxes[2], barplotaxes[3]])
    ax2.spines['bottom'].set_linewidth(linewidth[0])
    ax2.spines['left'].set_linewidth(linewidth[0])
    ax2.spines['top'].set_color("none")
    ax2.spines['right'].set_color("none")
    ax2.tick_params(width=2, labelsize = xyLabels[7])

    ax3 = plt.subplot(1, 3, 3)
    plt.scatter(np.random.normal(1, 0.05, len(gooddataRight)), gooddataRight, s=2, color = 'g')
    plt.scatter(np.random.normal(2, 0.05, len(gooddataLeft)), gooddataLeft, s=2, color = 'g')
    plt.scatter(np.random.normal(1, 0.05, len(baddataRight)), baddataRight, s=2, color = 'r', alpha = 0.2)
    plt.scatter(np.random.normal(2, 0.05, len(baddataLeft)), baddataLeft, s=2, color = 'r', alpha = 0.2)
    plt.scatter(1.2, np.mean(gooddataRight), s=25, color = 'g')
    plt.scatter(2.2, np.mean(gooddataLeft), s=25, color = 'g')
    plt.errorbar(1.2, np.mean(gooddataRight), yerr=scipy.stats.sem(gooddataRight))
    plt.errorbar(1.2, np.mean(gooddataLeft), yerr=scipy.stats.sem(gooddataLeft))
    plt.boxplot(gooddataRight, positions = [1.35])
    plt.boxplot(gooddataLeft, positions = [2.35])
    ax3.set_xlabel(xyLabels[2], fontsize = xyLabels[6])
    ax3.set_ylabel(xyLabels[3], fontsize = xyLabels[6])
    ax3.set_title(title[2], fontsize = title[3])
    ax3.set_xlim([scatterplotaxes[0], scatterplotaxes[1]])
    ax3.set_ylim([scatterplotaxes[2], scatterplotaxes[3]])
    plt.xticks([1,2], [xyLabels[4],xyLabels[5]], fontsize = xyLabels[6])
    ax3.spines['bottom'].set_linewidth(linewidth[0])
    ax3.spines['left'].set_linewidth(linewidth[0])
    ax3.spines['top'].set_color("none")
    ax3.spines['right'].set_color("none")
    ax3.tick_params(width=2, labelsize = xyLabels[7])

    plt.tight_layout(pad=0.5)
    return plt

In [ ]:
#run speed 
# this session 20/08 big is in Left - Big line is Left
for animal in animalList:
        sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
        for session in sessionList:
            plot_distribution(good_speedToRight[animal, session], good_speedToLeft[animal, session], bad_speedToRight[animal, session], bad_speedToLeft[animal, session], barplotaxes = [0, 100, 0, 1], maxminstepbin = [0,100,1], scatterplotaxes = [0.5, 2.5, 0, 100], xyLabels=["Speed cm/s","cumul %", "Direction of run", "Speed(cm/s)", "To Right", "To Left", 14, 12], title=["big = thick line = To Left "+animal, "", animal, 16], linewidth = [1.5])

plt.show()
print("########################################################################################################")
plot_distribution(good_speedToRightPooled, good_speedToLeftPooled, bad_speedToRightPooled, bad_speedToLeftPooled, barplotaxes = [0, 100, 0, 1], maxminstepbin = [0,100,1], scatterplotaxes = [0.5, 2.5, 0, 100], xyLabels=["Speed cm/s","cumul %", "Direction of run", "Speed(cm/s)", "To Right", "To Left", 14, 12], title=["Pooled"+"big = thick line = To Left ", "", "Pooled", 16], linewidth = [1.5])

In [ ]:
#stay duration 
for animal in animalList:
        sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
        for session in sessionList:
            plot_distribution(good_timeInRight[animal, session], good_timeInLeft[animal, session], bad_timeInRight[animal, session], bad_timeInLeft[animal, session], barplotaxes = [0, 5, 0, 1], maxminstepbin = [0,5,0.1], scatterplotaxes = [0.5, 2.5, 0, 30], xyLabels=["Time in zone (s)","%", "Zone", "Time in zone (s)", "In Right", "In Left", 14, 12], title=["big = thick line = In Left "+animal, "", animal, 16], linewidth = [1.5])


plt.show()
print("########################################################################################################")
plot_distribution(good_timeInRightPooled, good_timeInLeftPooled, bad_timeInRightPooled, bad_timeInLeftPooled, barplotaxes = [0, 5, 0, 1], maxminstepbin = [0,5,0.1],scatterplotaxes = [0.5, 2.5, 0, 30], xyLabels=["Time in zone (s)","%", "Zone", "Time in zone (s)", "In Right", "In Left", 14, 12], title=["Pooled"+"big = thick line = In Left", "", "Pooled", 16], linewidth = [1.5])

# bug time

In [ ]:
print(rawTime)

In [ ]:
a = np.diff(rawTime[animal, session])
def movinavg(interval, window_size):
    window = np.ones(int(window_size))/float(window_size)
    return np.convolve(interval, window, 'same')

In [ ]:
plt.plot(a, label = "data")
plt.plot(movinavg(a, 100), label="moving average")

plt.xlim(0, len(a))
plt.ylim(0, 0.1)
plt.xlabel("frame index")
plt.ylabel("time per frame (s)")
plt.legend()

In [ ]:
plt.plot(a, label = "data")
plt.plot(movinavg(a, 100), label="moving average")

plt.xlim(35000, len(a))
plt.ylim(0.03, 0.05)
plt.xlabel("frame index")
plt.ylabel("time per frame (s)")
plt.legend()

# test instant speed

In [ ]:
def extract_runsRightLeftFiltered(data, animal, session, boundary, limspeed=None):
    rawdata_run = {}
    rawdata_stay = {}
    good_speedRight_cms = []
    good_speedLeft_cms = []
    good_x_trackR = []
    good_x_trackL = []

    bad_speedRight_cms = []
    bad_speedLeft_cms = []
    bad_x_trackR = []
    bad_x_trackL = []
    all_speedRight_cms = [] 
    all_speedLeft_cms = []


    good_timeInRight = []
    good_timeInLeft = []
    good_x_stayR = []
    good_x_stayL = []

    bad_timeInRight = []
    bad_timeInLeft = []
    bad_x_stayR = []
    bad_x_stayL = []
    all_timeInRight = []
    all_timeInLeft = []
    testR = []
    testL = []
    good_testR = []
    bad_testR = []

    


    #crop the complete track at the set boundaries -> keep only the runs and set the rest to 0
    rawdata_run[animal, session] = [e/13.5 if e <= boundary[0]+50 and e>= boundary[1]-50 else 0 for e in data[animal, session]] #conversion pixel to cm maybe do this better 
    # use the split a list function to create a new sublist when there is a 0, which we just set up before
    for subsplit in split_a_list_at_zeros(rawdata_run[animal, session]):
        #check if the subsplit starts on the left or the right -> determine if the animal is running left or right
        if subsplit[0] < (((boundary[0]+boundary[1])/2) /13.5):
            #check if the subsplit is ending on the other side -> determine if this is a run 
            if subsplit[-1] > (((boundary[0]+boundary[1])/2) /13.5):
                #calculate distance run as the distance between first and last value
                distRight_cm = abs(subsplit[0]-subsplit[-1]) 
                #calculate time as number of values (frames) * time per frame
                timeRight = len(subsplit) * 0.05
                #check if there is a limspeed : if no compute all the runs -> used to compute limspeed
                #second pass : there is a computed limspeed, use it to sort good and bad runs
                if limspeed[0] is not None:
                    #check if this is a good or bad run and assign im to the good or bad array accordingly
                    if distRight_cm/timeRight > limspeed[0]:
                        #good run array
                        good_speedRight_cms.append(distRight_cm/timeRight)
                        good_x_trackR.append(subsplit)
                        good_testR.append(np.diff(subsplit))     
                    else:
                        #bad run array
                        bad_speedRight_cms.append(distRight_cm/timeRight)
                        bad_x_trackR.append(subsplit)
                        bad_testR.append(np.diff(subsplit))

                else:
                    #all runs array first pass
                    all_speedRight_cms.append(distRight_cm/timeRight)
                    testR.append(np.diff(subsplit))


        #same thing for the runs that go to the other side
        elif subsplit[0]>(((boundary[0]+boundary[1])/2)/13.5):
            if subsplit[-1]<(((boundary[0]+boundary[1])/2)/13.5):
                distLeft_cm = abs(subsplit[0]-subsplit[-1])
                timeLeft = len(subsplit) * 0.05
                if limspeed[1] is not None:
                    if distLeft_cm/timeLeft > limspeed[1]:
                        good_speedLeft_cms.append(distLeft_cm/timeLeft)   
                        good_x_trackL.append(np.diff(subsplit))
                    else:
                        bad_speedLeft_cms.append(distLeft_cm/timeLeft)
                        bad_x_trackL.append(np.diff(subsplit))
                else:
                    all_speedLeft_cms.append(distLeft_cm/timeLeft)
                    testL.append(np.diff(subsplit))

    # inverse crop, at the boundaries keep the distal parts, same process as above
    rawdata_stay[animal, session] = [e/13.5 if e >= boundary[0] or e<= boundary[1] else 0 for e in data[animal, session]]
    for subsplit in split_a_list_at_zeros(rawdata_stay[animal, session]):
        #first identify if the subsplit created is on the left or right by comparing to the middle
        if subsplit[0] > (((boundary[0]+boundary[1])/2) /13.5):
            # maybe add another filter i.e. out if less than 5 frames
            #same idea as above
            if limspeed[0] is not None:
                #add lenght (frames) of the subsplit * time per frame
                if len(subsplit)* 0.05 < limspeed[0]:
                    good_timeInRight.append(len(subsplit)* 0.05)#clean 
                    good_x_stayR.append(subsplit)
                else:
                    bad_timeInRight.append(len(subsplit)* 0.05)
                    bad_x_stayR.append(subsplit)
            else:
                all_timeInRight.append(len(subsplit)* 0.05)
                
        elif subsplit[0]<(((boundary[0]+boundary[1])/2)/13.5):
            #compute
            if limspeed[1] is not None:
                if len(subsplit)* 0.05 < limspeed[1]:
                    good_timeInLeft.append(len(subsplit)* 0.05)   
                    good_x_stayL.append(subsplit)
                else:
                    bad_timeInLeft.append(len(subsplit)* 0.05)
                    bad_x_stayL.append(subsplit)
            else:
                all_timeInLeft.append(len(subsplit)* 0.05)
   
    return good_speedRight_cms, good_speedLeft_cms, good_x_trackR, good_x_trackL, bad_speedRight_cms, bad_speedLeft_cms, bad_x_trackR, bad_x_trackL, all_speedRight_cms, all_speedLeft_cms, good_timeInRight, good_timeInLeft, good_x_stayR, good_x_stayL, bad_timeInRight, bad_timeInLeft, bad_x_stayR, bad_x_stayL, all_timeInRight, all_timeInLeft, testR, testL, good_testR, bad_testR

In [ ]:

testRight = {}
testLeft = {}
goodRight = {}
badRight = {}
for animal in animalList: 
        sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
        for session in sessionList:
            #extract all the runs to compute limspeed

            testRight[animal, session]= extract_runsRightLeftFiltered(rawPositionX, animal, session, boundary = params[animal, session]["boundaries"], limspeed = [None, None])[20]
            testLeft[animal, session] = extract_runsRightLeftFiltered(rawPositionX, animal, session, boundary = params[animal, session]["boundaries"], limspeed = [None, None])[21]
            goodRight[animal, session] = extract_runsRightLeftFiltered(rawPositionX, animal, session, boundary = params[animal, session]["boundaries"], limspeed = [limspeedToRight[animal, session], limspeedToLeft[animal, session]])[22]
            badRight[animal, session] = extract_runsRightLeftFiltered(rawPositionX, animal, session, boundary = params[animal, session]["boundaries"], limspeed = [limspeedToRight[animal, session], limspeedToLeft[animal, session]])[23]

            print(animal, "done")

In [ ]:

for i in testRight[animal, session]:
    plt.plot(np.linspace(0, len(i), len(i)), i, color='g' , linewidth=0.3)
plt.xlim(0, 50)
plt.ylim(-10, 10)

In [ ]:
for i in goodRight[animal, session]:
    plt.plot(np.linspace(0, len(i), len(i)), i, color='g' , linewidth=0.3)
for i in badRight[animal, session]:
    plt.plot(np.linspace(0, len(i), len(i)), i, color='r' , linewidth=0.3)
plt.xlim(0, 50)
plt.ylim(-10, 10)

In [ ]:
for i in goodRight[animal, session]:
    plt.plot(np.linspace(0, len(i), len(i)), i, color='g' , linewidth=0.3)
for i in badRight[animal, session]:
    plt.plot(np.linspace(0, len(i), len(i)), i, color='r' , linewidth=0.3)
plt.xlim(0, 50)
plt.ylim(-1, 5);

# TODOLIST


In [ ]:
###TODO###
#convert position to cm cleaner
#comments
#explain how data is processed
#explore weight effect and evolution /speed /peakspeed /number of runs
#do something with number of runs
#look for instant speed inside the identified runs --> see if both methods give the same result
#look for peak speed
#!!!!!!!evolution of speed in the session
#smooth position ?
#runs +extremities on one plot
#generate html/pdf recap+stats
#pickle useful data
#compare between days
#lickdata
#lick data --> waiting time
#add stats on figs
#noise videotrack / improve videotrack
#Investigate change in speed during a single session (hypothese [case reward size is the same for leftward or rightward trials] : speed should go down ?; [case reward size is diffrent for leftward and rightward trials]: does speed slow down more quickly for left or right runs ?
#test median
#BIG/small on figs
#problem with time............
#average track per animal/group 
#specify analysis groups for stats
#analyse training data
#analyse /2 data
#clean graphes for talk 
#stats timeBins


##DONE##
#######see git log for DONE